## Monte Carlo Tree Search


In [32]:
import numpy as np
import math  
import random
import time
import operator
from colorama import Fore, Style


In [33]:
class Node():
    def __init__(self, state, parent):
        self.N = 0 #Total number of visits
        self.Q = 0 #Total accumulated reward
        self.is_end_state = state.is_end_state()
        self.is_expanded_completly = self.is_end_state
        self.state = state 
        self.parent = parent
        self.children = {}

$UCT = \frac{Q}{N} + c\sqrt{\frac{\log{N}}{N}}$


In [34]:
class monte_carlo_tree_search():
    def __init__(self, c = 2):
        self.c = c   # how much to explore
 
    def rollout_policy(self, state):## randomly pick
        while state.is_end_state()!= True:
            _available_actions = state.available_actions()
            _random_action = random.choice(_available_actions)
            state = state.play_action(_random_action)
        return state.get_reward()

    # Based on a simulation policy, we continue playing from the expansion node until a terminal node is reached
    # select unvisited child nodes
    def simulate(self):
        _node  = self.select_unvisited_nodes(self.root)
        _Q = self.rollout_policy(_node.state)
        self.backpropagate(node=_node, _Q = _Q)



    def select_unvisited_nodes(self, node : Node):
        while node.is_end_state ==False:
            if node.is_expanded_completly == True:
                node = self.pick_child(node, self.c)
            else:
                #expanding node
                return self.expand(node)
        return node

    def traverse(self, start_node):
        self.root = Node(start_node, None)
        tme = time.time() + 2
        while time.time() < tme:
            self.simulate()

        bestChilds = self.pick_child(self.root, c = 0)
        action=(action for action, node in self.root.children.items() if node is bestChilds).__next__()
        return action

    def expand(self,node):
        actions = node.state.available_actions()
        for action in actions:
            if action not in node.children:
                new_node = Node(node.state.play_action(action), node)
                node.children[action] = new_node
                if len(actions) == len(node.children):
                    node.is_expanded_completly = True
                return new_node


    def calculate_value(self,node,child,c):
         return node.state.player * ( child.Q / child.N ) + c * np.sqrt(  np.log(node.N) / child.N )        

    def calculate_values(self, node, c):
        max_value = - math.inf
        good_options = []
        for child in node.children.values():
            uct =self.calculate_value(node,child,c)
            if uct > max_value:
                max_value = uct
                good_options = [child]
            elif uct == max_value:
                good_options.append(child)
        return good_options
    
    def pick_child(self, node, c):
        return random.choice(self.calculate_values(node, c))

    # update
    def backpropagate(self,node : Node, _Q):
        while node is not None:
            node.N += 1
            node.Q += _Q
            node = node.parent



In [35]:
class Action():
    def __init__(self, player, x, y):
        self.player = player
        self.x = x
        self.y = y
    
    ## Just add to remove hash error
    def __str__(self):
        return str((self.x, self.y))

    def __repr__(self):
        return str(self)

    def __eq__(self, other):
        return self.__class__ == other.__class__ and self.x == other.x and self.y == other.y and self.player == other.player

    def __hash__(self):
        return hash((self.x, self.y, self.player))

In [36]:
from copy import deepcopy
from functools import reduce


In [37]:
class Tic_Tac_Toe():
    def __init__(self, size=3):
        self.size = size
        self.board = (np.zeros((size, size))).tolist()
        self.player = 1  

    def available_actions(self):
        actions = []
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    actions.append(Action(self.player, x=i, y=j))
        return actions
    
    def play_action(self, action : Action):
        new_state = deepcopy(self)
        new_state.board[action.x][action.y] = action.player
        new_state.player = self.player * -1
        return new_state
    
    # Terminal nodes
    def is_end_state(self):
        # first chekc row
        for row in self.board:
            if abs(sum(row)) == self.size:
                return True
        # second check column
        for column in np.transpose(self.board):
            if abs(sum(column)) == self.size:
                return True
        # check for diagonals
        if abs(sum(np.diag(self.board))) == self.size or abs(sum(np.diag(np.rot90(self.board)))) == self.size:
            return True

        return reduce(operator.mul, sum(self.board, []), 1)
    
    def print_board(self):
        for i in self.board:
            for j in i:
                if j == -1: print(Fore.GREEN + 'O' + Style.RESET_ALL, end='\t') # for first player
                elif j == 1: print(Fore.BLUE + 'X' + Style.RESET_ALL, end='\t') # for second playe
                else: print('-', end='\t')
            print("")
        print('____________________________\n')

    # -1 for O and 1 for X
    def get_reward(self):
        for row in self.board:
            if abs(sum(row)) == self.size:
                return sum(row) / self.size
        for column in list(map(list, zip(*self.board))):
            if abs(sum(column)) == self.size:
                return sum(column) / self.size
        for diagonal in [[self.board[i][i] for i in range(len(self.board))], [self.board[i][len(self.board) - i - 1] for i in range(len(self.board))]]:
            if abs(sum(diagonal)) == self.size:
                return sum(diagonal) / self.size
        return False
  


AIvsAI

In [38]:
mcts = monte_carlo_tree_search(c = 0.8 )
game = Tic_Tac_Toe()
while game.is_end_state() == False:
    action = mcts.traverse(start_node=game)
    if game.player == 1:
      print("X --> "+str(action) + "\n")
    else:
      print("O --> "+str(action) +" \n")
    game = game.play_action(action)
    game.print_board()


X --> (1, 1)

-	-	-	
-	X	-	
-	-	-	
____________________________

O --> (2, 0) 

-	-	-	
-	X	-	
O	-	-	
____________________________

X --> (2, 1)

-	-	-	
-	X	-	
O	X	-	
____________________________

O --> (0, 1) 

-	O	-	
-	X	-	
O	X	-	
____________________________

X --> (1, 0)

-	O	-	
X	X	-	
O	X	-	
____________________________

O --> (1, 2) 

-	O	-	
X	X	O	
O	X	-	
____________________________

X --> (0, 0)

X	O	-	
X	X	O	
O	X	-	
____________________________

O --> (2, 2) 

X	O	-	
X	X	O	
O	X	O	
____________________________

X --> (0, 2)

X	O	X	
X	X	O	
O	X	O	
____________________________



Human VS AI

In [28]:

mcts = monte_carlo_tree_search(c=0.8)
game = Tic_Tac_Toe()
flag_turn = 'x'

while game.is_end_state() == False:
    if flag_turn == 'x':
        turn = 1
        while turn == 1:
            x, y = input(
                'Enter your move "row,column": ').split(',')
            x, y = int(x), int(y)
            validActions = game.available_actions()
            action = Action(1, x, y)
            print("X (human) --> "+ str(action) + "\n" )
            game = game.play_action(action)
            game.print_board()
            flag_turn = 'o'
            turn = 2
    else:
        action = mcts.traverse(start_node=game)
        print("O (AI) --> "+str(action)+" \n" )
        game = game.play_action(action)
        game.print_board()
        flag_turn = 'x'

X (human) -> (0, 2) 

-	-	X	
-	-	-	
-	-	-	
____________________________

O (AI) ->  (1, 1) 

-	-	X	
-	O	-	
-	-	-	
____________________________

X (human) -> (1, 2) 

-	-	X	
-	O	X	
-	-	-	
____________________________

O (AI) ->  (2, 2) 

-	-	X	
-	O	X	
-	-	O	
____________________________

X (human) -> (2, 1) 

-	-	X	
-	O	X	
-	X	O	
____________________________

O (AI) ->  (0, 0) 

O	-	X	
-	O	X	
-	X	O	
____________________________



5x5

In [40]:
mcts = monte_carlo_tree_search(c=1)
game = Tic_Tac_Toe(size=5)
flag = 'O'
while game.is_end_state() == False:

    if flag == 'x':
        turn = 1
        while turn == 1:
            x, y = input(
                'Human Enter your move "row,column": ').split(',')
            x, y = int(x), int(y)
            validActions = game.available_actions()
            action = Action(-1, x, y)

            print("Human --> "+str(action)+" \n")
            game = game.play_action(action)
            game.print_board()
            flag = 'o'
            turn = 2

    else:
        action = mcts.traverse(start_node=game)
        print("AI --> "+str(action)+" \n")
        game = game.play_action(action)
        game.print_board()
        flag = 'x'


AI --> (4, 4) 

-	-	-	-	-	
-	-	-	-	-	
-	-	-	-	-	
-	-	-	-	-	
-	-	-	-	X	
____________________________

Human --> (0, 0) 

O	-	-	-	-	
-	-	-	-	-	
-	-	-	-	-	
-	-	-	-	-	
-	-	-	-	X	
____________________________

AI --> (2, 3) 

O	-	-	-	-	
-	-	-	-	-	
-	-	-	X	-	
-	-	-	-	-	
-	-	-	-	X	
____________________________

Human --> (0, 1) 

O	O	-	-	-	
-	-	-	-	-	
-	-	-	X	-	
-	-	-	-	-	
-	-	-	-	X	
____________________________

AI --> (4, 2) 

O	O	-	-	-	
-	-	-	-	-	
-	-	-	X	-	
-	-	-	-	-	
-	-	X	-	X	
____________________________

Human --> (2, 2) 

O	O	-	-	-	
-	-	-	-	-	
-	-	O	X	-	
-	-	-	-	-	
-	-	X	-	X	
____________________________

AI --> (4, 0) 

O	O	-	-	-	
-	-	-	-	-	
-	-	O	X	-	
-	-	-	-	-	
X	-	X	-	X	
____________________________

Human --> (0, 3) 

O	O	-	O	-	
-	-	-	-	-	
-	-	O	X	-	
-	-	-	-	-	
X	-	X	-	X	
____________________________

AI --> (4, 1) 

O	O	-	O	-	
-	-	-	-	-	
-	-	O	X	-	
-	-	-	-	-	
X	X	X	-	X	
____________________________

Human --> (1, 0) 

O	O	-	O	-	
O	-	-	-	-	
-	-	O	X	-	
-	-	-	-	-	
X	X	X	-	X	
_____